## 算术运算

### 数据对齐，填充值
不同索引的对象之间可以进行算术运算。
- 若存在不同的索引项，运算结果的索引是运算对象的索引的并集，而在两个对象中并不同时存在的索引处值为空值NaN；
- 一个运算对象使用add方法传入另外一个运算对象和fill_value参数，运算对象在另外一个对象中不存在的索引处值填入参数fill_value的值然后再进行运算；
- 类似add方法，还有sub、div、floordiv、mul，pow方法，若在前面加上字母“r”，则表示翻转参数，比如被减数和减数身份对换；
- fill_value参数同样适用于reindex重新索引；

In [1]:
import numpy as np
import pandas as pd
frame1=pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]],columns=list('ABC'),index=list('bce'))
frame2=pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]],columns=list('ACD'),index=list('abc'))

In [2]:
frame1

,A,B,C
b,1,2,3
c,4,5,6
e,7,8,9


In [3]:
frame2

,A,C,D
a,1,2,3
b,4,5,6
c,7,8,9


In [4]:
# 相加后，行索引和列索引为frame1和frame2的行、列索引的并集
# 在frame1和frame2同时存在的索引位置的数值为运算结果，其他均为NaN
frame1+frame2

,A,B,C,D
a,NaN,NaN,NaN,NaN
b,5.0,NaN,8.0,NaN
c,11.0,NaN,14.0,NaN
e,NaN,NaN,NaN,NaN


In [5]:
frame1.add(frame2,fill_value=0) # 使用add方法，可以在不存在的索引处填入值

,A,B,C,D
a,1.0,NaN,2.0,3.0
b,5.0,2.0,8.0,6.0
c,11.0,5.0,14.0,9.0
e,7.0,8.0,9.0,NaN


In [6]:
frame1.sub(frame2,fill_value=0) # 减法，frame1-frame2

,A,B,C,D
a,-1.0,NaN,-2.0,-3.0
b,-3.0,2.0,-2.0,-6.0
c,-3.0,5.0,-2.0,-9.0
e,7.0,8.0,9.0,NaN


In [7]:
frame1.rsub(frame2,fill_value=0) # rsub，翻转参数，frame2-frame1

,A,B,C,D
a,1.0,NaN,2.0,3.0
b,3.0,-2.0,2.0,6.0
c,3.0,-5.0,2.0,9.0
e,-7.0,-8.0,-9.0,NaN


In [8]:
frame1.reindex(index=frame2.index) # 使用frame2的行索引对frame1重新索引，缺失内容为空

,A,B,C
a,NaN,NaN,NaN
b,1.0,2.0,3.0
c,4.0,5.0,6.0


In [9]:
frame1.reindex(index=frame2.index,fill_value=0) # reindex也可以使用fill_value参数填充缺失内容

,A,B,C
a,0,0,0
b,1,2,3
c,4,5,6


### DataFrame和Series之间运算
- DataFrame和Series之间运算，将Series的索引匹配到DataFrame的列，并一行行向下广播；
- 若有缺失的索引，会重新索引生成索引的并集；
- 若需要将Series索引匹配到DataFrame的行，并一在列上广播，必须使用算术运算方法，且在参数里指定轴为行索引axis='index'或axis=0，算术方法前加字母“r”可以翻转参数；

In [10]:
frame1

,A,B,C
b,1,2,3
c,4,5,6
e,7,8,9


In [11]:
s1=frame1.loc['b'] # 选取frame1中标签为b的行作为s1
s1

A    1
B    2
C    3
Name: b, dtype: int64

In [12]:
frame1-s1 # s1的索引和frame1的列匹配，并逐行进行运算

,A,B,C
b,0,0,0
c,3,3,3
e,6,6,6


In [13]:
s2=s1.copy() # 使用s1副本生成s2
s2.index=list('BAD') # 对s2重新索引，使之存在frame1中没有的索引项，并改变已存在的索引项顺序
s2

B    1
A    2
D    3
Name: b, dtype: int64

In [14]:
frame1-s2 # 根据对应的索引标签值来运算而非位置，缺失的索引项被加入索引之中，且值为空

,A,B,C,D
b,-1.0,1.0,NaN,NaN
c,2.0,4.0,NaN,NaN
e,5.0,7.0,NaN,NaN


In [15]:
s3=frame1['B'] # 获取frame1的B列作为s3
s3

b    2
c    5
e    8
Name: B, dtype: int64

In [16]:
frame1.sub(s3,axis='index') # 匹配行，一列一列进行运算，必须使用算术方法，参数指定轴为行索引index

,A,B,C
b,-1,0,1
c,-1,0,1
e,-1,0,1


In [17]:
frame1.rsub(s3,axis='index') # 同样方法名前面加上“r”可以翻转参数

,A,B,C
b,1,0,-1
c,1,0,-1
e,1,0,-1


## 函数应用和映射
- DataFrame自带有许多常见的如sum，mean等，可以直接使用。但如果要使用其他的函数或自定义的函数，则需要使用apply方法来调用这些函数；
- apply方法适用于元素级数组函数，将所调用的函数应用到DataFrame的每列（默认）或者每行所形成的一维数组上，得到一个以列/行标签为索引的Series，使用asix参数可以指定轴为列还是行；
- axis='index'表示逐列应用函数计算每行的元素，axis='columns'表示逐行应用函数计算每列的元素；
- applymap方法适用于元素级函数，将调用的函数应用到DataFrame里的每个元素。

In [18]:
frame=pd.DataFrame(np.arange(1,10).reshape(3,3),index=list('abc'),columns=list('ABC'))
frame

,A,B,C
a,1,2,3
b,4,5,6
c,7,8,9


In [19]:
def z(i): # 自定义一个函数，对所有元素求和并返回值
    return i.sum()

In [20]:
frame.apply(z) # 使用apply调用z函数，默认将z函数应用到每列上，得到一个索引为列索引的Series

A    12
B    15
C    18
dtype: int64

In [21]:
frame.apply(z,axis='columns') # 加入axis参数，指定轴

a     6
b    15
c    24
dtype: int64

In [22]:
def x(i): # 元素自己相加并返回值
    return i+i

In [23]:
frame.applymap(x) # applymap将函数应用到每个元素

,A,B,C
a,2,4,6
b,8,10,12
c,14,16,18


## 排序和排名
### 排序
`sort_index`：根据标签排序  
`sort_values`：根据值排序  
  
参数  
- `axis`：指定轴的标签进行排序，默认0 ，在列方向对每行值排序
- `ascending`：排列顺序，默认True正序排列，False倒序排列  
- `kind`：排序类型，`quicksort`（默认），`mergesort`，`heapsort`  
- `na_position`：空值NaN位置，默认`last`空值放最后，`first`空值放最前
- `by`：仅`sort_values`有，指定哪个标签下的值参与排序，axis=0选取列标签，axis=1选取行标签，要同时对多列或者多行排序请以列表['a','c']形式传入

In [25]:
frame=pd.DataFrame(np.arange(16).reshape(4,4),index=list('dbca'),columns=list('BCAD'))
frame

,B,C,A,D
d,0,1,2,3
b,4,5,6,7
c,8,9,10,11
a,12,13,14,15


In [26]:
frame.sort_index() # 根据标签排序，默认以行标签值排序

,B,C,A,D
a,12,13,14,15
b,4,5,6,7
c,8,9,10,11
d,0,1,2,3


In [27]:
frame.sort_index(axis='columns',ascending=False) # 根据列标签值倒序排列

,D,C,B,A
d,3,1,0,2
b,7,5,4,6
c,11,9,8,10
a,15,13,12,14


In [28]:
frame['C']['c']=np.nan # 给frame添加个NaN值
frame

,B,C,A,D
d,0,1.0,2,3
b,4,5.0,6,7
c,8,NaN,10,11
a,12,13.0,14,15


In [30]:
frame.sort_values(by='C') # 对C列进行排序，默认正序，空值放最后

,B,C,A,D
d,0,1.0,2,3
b,4,5.0,6,7
a,12,13.0,14,15
c,8,NaN,10,11


In [31]:
frame.sort_values(axis=1,by='c',ascending=False,na_position='first') # 对c行进行倒序排序，空值放最前

,C,D,A,B
d,1.0,3,2,0
b,5.0,7,6,4
c,NaN,11,10,8
a,13.0,15,14,12


In [32]:
frame.sort_values(by=['A','C']) # 对多列进行排序

,B,C,A,D
d,0,1.0,2,3
b,4,5.0,6,7
c,8,NaN,10,11
a,12,13.0,14,15


### 排名
`rank`：从1开始对对象中的有效数据进行排名，遇到平级（相等）数据默认使用该数据的平均排名，比如两个相同数字分别排名5，6位，平均排名就是(5+6)/2=5.5位 
  
`method`参数
- `average`：默认，平均排名
- `first`：相同数据不进行平均排名，以数据出现的顺序进行排名
- `max` `min`：相同数据使用最大/最小的排名，若排名5、6位，`max`选取6，`min`选取5
- `dense`：类似`min`，但排名相同的数据只占用一个排名位，后续的排名以排名位+1计算，如5、6位`min`选取排名位5，后面的排名位跳过6从7开始，而`dense`后面的排名位置从6开始
- `rank`应用到DataFrame上，默认axis=0针对每列进行排名，axis=1针对每行进行排名

In [34]:
series=pd.Series([5,7,3,0,-2,3,7])
series

0    5
1    7
2    3
3    0
4   -2
5    3
6    7
dtype: int64

In [35]:
series.rank() # 2个数值3分别排名3，4位，平均排名就是3.5位，数值7也是同理

0    5.0
1    6.5
2    3.5
3    2.0
4    1.0
5    3.5
6    6.5
dtype: float64

In [36]:
series.rank(method='first') # first，相同数值以出现顺序进行排序

0    5.0
1    6.0
2    3.0
3    2.0
4    1.0
5    4.0
6    7.0
dtype: float64

In [37]:
series.rank(method='min') # 2个数值3分别排名3，4位，使用min选取最小的3，第4位被跳过，后面的数值5排名第5位

0    5.0
1    6.0
2    3.0
3    2.0
4    1.0
5    3.0
6    6.0
dtype: float64

In [38]:
series.rank(method='dense') # 2个数值3分别排名3，4位，选取最小的3，后面的数值5排名第4位

0    4.0
1    5.0
2    3.0
3    2.0
4    1.0
5    3.0
6    5.0
dtype: float64

In [39]:
frame=pd.DataFrame([[3,2,5],[6,3,5],[8,2,1]],index=list('abc'),columns=list('ABC'))
frame

,A,B,C
a,3,2,5
b,6,3,5
c,8,2,1


In [40]:
frame.rank() # 默认针对每列进行排名

,A,B,C
a,1.0,1.5,2.5
b,2.0,3.0,2.5
c,3.0,1.5,1.0


In [41]:
frame.rank(axis=1) # 针对每行排名

,A,B,C
a,2.0,1.0,3.0
b,3.0,1.0,2.0
c,3.0,2.0,1.0
